In [26]:
import pandas as pd
import numpy as np
import pulp


In [5]:
df = pd.read_csv("data/nutrition.csv",index_col=0)
df.head()

,descriptor,calories,carbohydrates,fat,fiber,protein,sodium,Breakfast,Lunch,Dinner,Snack,servings,totalTime,cookTime
index,,,,,,,,,,,,,,
0,15-minute-vegetable-curry,97.00,20.00,1.00,6.00,5.00,0.44800,0,1,1,0,5.0,15.0,10.0
1,16-brunch-recipes-for-easter-and-beyond,119.05,0.55,10.15,0.40,6.25,0.16705,0,0,0,0,2.0,20.0,15.0
2,20-minute-creamy-spinach-artichoke-pasta,369.63,54.60,11.10,6.53,14.83,1.07975,0,0,0,0,4.0,20.0,15.0
3,20-minute-honey-mustard-chicken,299.65,8.93,10.70,0.05,38.28,0.58900,0,0,1,0,4.0,20.0,15.0
4,22-fast-easy-one-pot-meals,484.00,12.00,40.00,5.00,20.00,1.17800,0,1,1,0,4.0,15.0,15.0


In [6]:
d = pd.DataFrame()
d['n'] = df.descriptor
d['1'] = pd.Series(['beverage', 'snack', 'breakfast', 'beverage', 'dessert', 'breakfast', 'breakfast', 'snack', 'snack', 'snack', 'breakfast', 'lunch', 'snack', 'beverage', 'dinner', 'lunch', 'lunch', 'dessert', 'snack', 'snack', 'snack', 'beverage', 'breakfast', 'snack'])
d['2'] = pd.Series(['beverage','snack','breakfast','beverage','desert','breakfast','breakfast','snack','snack','snack','breakfast','lunch','snack','beverage','lunch','beverage','snack','desert','beverage','snack','snack','beverage','desert','breakfast','snack'])
d['3'] = pd.Series(['beverage', 'snack', 'breakfast', 'beverage', 'desert', 'breakfast', 'breakfast', 'snack', 'snack', 'snack', 'breakfast', 'lunch', 'snack', 'beverage', 'lunch', 'beverage', 'lunch', 'breakfast', 'desert', 'beverage', 'snack', 'snack', 'beverage', 'breakfast', 'snack'])
d

,n,1,2,3
index,,,,
0,15-minute-vegetable-curry,beverage,beverage,beverage
1,16-brunch-recipes-for-easter-and-beyond,snack,snack,snack
2,20-minute-creamy-spinach-artichoke-pasta,breakfast,breakfast,breakfast
3,20-minute-honey-mustard-chicken,beverage,beverage,beverage
4,22-fast-easy-one-pot-meals,dessert,desert,desert
...,...,...,...,...
1146,zucchini-fritters,NaN,NaN,NaN
1147,zucchini-orzo-salad-chimichurri,NaN,NaN,NaN
1148,zucchini-pizza-boats,NaN,NaN,NaN


## Optimization fun

In [101]:
shortDf = df.sample(n=5,axis=0)

constraints = pd.DataFrame(index=shortDf.columns.unique())
constraints['min'] = {'calories': 2100, 'fat': 50, 'Lunch': 1, 'Dinner': 1, 'Breakfast': 1, 'protein': 70}
constraints['max'] = {'calories': 2800, 'fat': 120, 'protein': 150}
constraints.head()

,min,max
descriptor,NaN,NaN
calories,2100.0,2800.0
carbohydrates,NaN,NaN
fat,50.0,120.0
fiber,NaN,NaN


In [102]:
diet_model = pulp.LpProblem("The_Diet_Problem")


X_dict = pulp.LpVariable.dict("x_%s", shortDf.descriptor, lowBound = 0, upBound = 2, cat=pulp.LpInteger)
X = list(X_dict.values())

# diet_model += shortDf.calories.dot(X) >= 1800
# diet_model += shortDf.calories.dot(X) <= 2800
# diet_model += shortDf.protein.dot(X) >= 130
# diet_model += shortDf.fat.dot(X) >= 58
# diet_model += df.Cholesterol.dot(X) <= 1
for varName, varConstraints in constraints.iterrows():
    if varConstraints['min'] > 0 and varName in shortDf.columns:
        diet_model += shortDf[varName].dot(X) >= varConstraints['min']
        
    if varConstraints['max'] > 0:
        diet_model += shortDf[varName].dot(X) <= varConstraints['max']

diet_model

The_Diet_Problem:
MINIMIZE
None
SUBJECT TO
_C1: 139.55 x_garlic_parmesan_roasted_cauliflower
 + 222.62 x_slow_cooker_chicken_ropa_vieja + 544.65 x_teriyaki_pork_sandwiches
 + 472.5 x_tilapia_fish_sticks + 432 x_tuna_noodle_casserole >= 2100

_C2: 139.55 x_garlic_parmesan_roasted_cauliflower
 + 222.62 x_slow_cooker_chicken_ropa_vieja + 544.65 x_teriyaki_pork_sandwiches
 + 472.5 x_tilapia_fish_sticks + 432 x_tuna_noodle_casserole <= 2800

_C3: 9.1 x_garlic_parmesan_roasted_cauliflower
 + 4.3 x_slow_cooker_chicken_ropa_vieja + 23.18 x_teriyaki_pork_sandwiches
 + 19.7 x_tilapia_fish_sticks + 20 x_tuna_noodle_casserole >= 50

_C4: 9.1 x_garlic_parmesan_roasted_cauliflower
 + 4.3 x_slow_cooker_chicken_ropa_vieja + 23.18 x_teriyaki_pork_sandwiches
 + 19.7 x_tilapia_fish_sticks + 20 x_tuna_noodle_casserole <= 120

_C5: 5.85 x_garlic_parmesan_roasted_cauliflower
 + 35.52 x_slow_cooker_chicken_ropa_vieja + 37.59 x_teriyaki_pork_sandwiches
 + 31.83 x_tilapia_fish_sticks + 24 x_tuna_noodle_cassero

In [99]:
diet_model.solve()

-1

In [92]:
X

[x_cheese_enchiladas,
 x_roasted_cauliflower_and_quinoa_salad,
 x_garlic_toast_balsamic_tomatoes_white_beans]